In [2]:
import boto3

In [4]:
s3 = boto3.resource('s3',
    region_name='us-west-2')

In [5]:
try:
    s3.create_bucket(Bucket='bucket-14828-hw2-yingdong', CreateBucketConfiguration={
    'LocationConstraint': 'us-west-2'}) 
except Exception as e:
    print (e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [6]:
bucket = s3.Bucket("bucket-14828-hw2-yingdong")
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'GCY3NFARMEY0F0B6',
  'HostId': '/qLVOEY6LQ5l4sA3ykQ6pgdJn0Xl35T7GPSdpcuiwPq6GkAid8egFzyNqP9bjqLS5Cim0etkmTA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/qLVOEY6LQ5l4sA3ykQ6pgdJn0Xl35T7GPSdpcuiwPq6GkAid8egFzyNqP9bjqLS5Cim0etkmTA=',
   'x-amz-request-id': 'GCY3NFARMEY0F0B6',
   'date': 'Tue, 19 Oct 2021 23:45:18 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [6]:
body = open('./exp1.csv', 'rb')

In [7]:
o = s3.Object('bucket-14828-hw2-yingdong', 'test').put(Body=body )

In [8]:
s3.Object('bucket-14828-hw2-yingdong', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'P7404X0KFZZPRZMQ',
  'HostId': 'LafaZFpHLK0D0qWHq/+fFEzWB14FvRr7Zc87ebhAizfgDRKwH/HV7X+Y5ZCeXoS29BxIO0fsbec=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LafaZFpHLK0D0qWHq/+fFEzWB14FvRr7Zc87ebhAizfgDRKwH/HV7X+Y5ZCeXoS29BxIO0fsbec=',
   'x-amz-request-id': 'P7404X0KFZZPRZMQ',
   'date': 'Mon, 18 Oct 2021 22:14:06 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [7]:
dyndb = boto3.resource('dynamodb',
    region_name='us-west-2')

In [16]:
try:
    table = dyndb.create_table(
        TableName='DataTable', 
        KeySchema=[
                {
                    'AttributeName': 'PartitionKey',
                    'KeyType': 'HASH'
                }, 
                {
                    'AttributeName': 'RowKey',
                    'KeyType': 'RANGE'
                }
        ], 
        AttributeDefinitions=[
                {
                    'AttributeName': 'PartitionKey',
                    'AttributeType': 'S'
                }, 
                {
                    'AttributeName': 'RowKey',
                    'AttributeType': 'S'
                },
        ], 
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print (e)
    #if there is an exception, the table may already exist.
    table = dyndb.Table("DataTable")

In [17]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [18]:
print(table.item_count)

0


In [19]:
import csv

In [31]:
with open('./experiments.csv', 'rt') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)
        body = open(item[4], 'rb')
        s3.Object('bucket-14828-hw2-yingdong', item[4]).put(Body=body)
        url = " https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/"+item[4]
        metadata_item = {'PartitionKey': item[4], 'RowKey': item[0],
                'Temp' : item[1], 'Conductivity' : item[2], 'Concentration': item[3],'url':url}
        try: 
            table.put_item(Item=metadata_item)
        except:
            print('item may already be there or another failure')

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [10]:
response = table.get_item( Key={
        'PartitionKey': 'exp1.csv',
        'RowKey': '1'
    }
)
item = response['Item'] 
print(item)

{'Temp': '-1', 'RowKey': '1', 'Conductivity': '52', 'Concentration': '3.4', 'PartitionKey': 'exp1.csv', 'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp1.csv'}


In [33]:
response

{'Item': {'Temp': '-1',
  'RowKey': '1',
  'Conductivity': '52',
  'Concentration': '3.4',
  'PartitionKey': 'exp1.csv',
  'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp1.csv'},
 'ResponseMetadata': {'RequestId': '9FGP3S8EQKPOEHEIM2ENK339B3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 18 Oct 2021 23:06:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '218',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9FGP3S8EQKPOEHEIM2ENK339B3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '657836644'},
  'RetryAttempts': 0}}

In [11]:
response = table.get_item( Key={
        'PartitionKey': 'exp2.csv',
        'RowKey': '2'
    }
)
item = response['Item'] 
print(item)

{'Temp': '-2', 'RowKey': '2', 'Conductivity': '52.1', 'Concentration': '3.4', 'PartitionKey': 'exp2.csv', 'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp2.csv'}


In [12]:
response

{'Item': {'Temp': '-2',
  'RowKey': '2',
  'Conductivity': '52.1',
  'Concentration': '3.4',
  'PartitionKey': 'exp2.csv',
  'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp2.csv'},
 'ResponseMetadata': {'RequestId': '3U6GQ6AFTTTEE753SRVACAAUANVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 19 Oct 2021 23:48:59 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '220',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3U6GQ6AFTTTEE753SRVACAAUANVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3997739013'},
  'RetryAttempts': 0}}

In [13]:
response = table.get_item( Key={
        'PartitionKey': 'exp3.csv',
        'RowKey': '3'
    }
)
item = response['Item'] 
print(item)

{'Temp': '-2.93', 'RowKey': '3', 'Conductivity': '57.1', 'Concentration': '3.7', 'PartitionKey': 'exp3.csv', 'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp3.csv'}


In [14]:
response

{'Item': {'Temp': '-2.93',
  'RowKey': '3',
  'Conductivity': '57.1',
  'Concentration': '3.7',
  'PartitionKey': 'exp3.csv',
  'url': ' https://s3-us-west-2.amazonaws.com/bucket-14828-hw2-yingdong/exp3.csv'},
 'ResponseMetadata': {'RequestId': 'MNC5JQNUOLFGT4TG0RV32JIG8JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 19 Oct 2021 23:52:28 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '223',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MNC5JQNUOLFGT4TG0RV32JIG8JVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3655119686'},
  'RetryAttempts': 0}}